In [25]:
library('igraph')
g <- read.graph('./data/movie_edge_weight.txt', format='ncol', directed=FALSE)


Attaching package: ‘igraph’

The following objects are masked from ‘package:stats’:

    decompose, spectrum

The following object is masked from ‘package:base’:

    union



In [30]:
#genreate genre
movie_genre_file <- file("./data/movieID_genre.txt", open="r")
line <- readLines(movie_genre_file, 1, encoding="latin1")
genre_list <- rep("", vcount(g))
while(length(line) != 0) {
    line <- strsplit(line,"\t\t")
    genre <- line[[1]][3]
    movie_id <- line[[1]][2]
    node_id <- V(g)[V(g)$name == movie_id]
    genre_list[node_id] <- genre
    line <- readLines(movie_genre_file, 1, encoding="latin1")
} 
close(movie_genre_file)

In [31]:
save(genre_list, file = "./data/genreList.RData")

In [32]:
# index the nodes
V(g)$idx <- 1:vcount(g)

V(g)$genre <- genre_list

In [33]:
write_graph(g, "./data/movie_genre_graph.txt", format = "edgelist")

In [ ]:
# remove the movies which have no genres
vertices <- V(g)[genre_list != ""]
g.genre <- induced_subgraph(g, vertices, impl = "copy_and_delete")

# Fast Greedy community detection
g.genre.fgc <- cluster_fast_greedy(g.genre)

In [ ]:
write_graph(g.genre, "./data/movie_genre_subgraph.txt", format = "edgelist")

In [ ]:
# generate n random integers
n <- 10
communities <- sample(1:length(g.genre.fgc), n, replace = FALSE)

# compute the frequency of each genre in a community
for (i in communities) {
    genre_list <- V(g.genre)[g.genre.fgc[[i]]]$genre
    genre.table <- table(genre_list)
    genre.freq <- prop.table(genre.table)
    
    cat("=======Community", i, "=======\n")
    print(genre.table)
    print(genre.freq)

    title = sprintf("Genre Distribution in Community %d", i)
    barplot(genre.freq, main = title, ylab = "Frequency", xlab = "Genre")
#     genre_set <- names(genre.table)
#     count <- as.vector(genre.table)
#     freq <- as.vector(genre.freq)

}